## Notes
29-Jun-18: Last run today to help prepare for the meeting with Mark Herbster

Run in '/home/ubuntu/msc-project-speaker-recognition/msc-project'

on aws-big


In [1]:
import os
import sys

os.chdir('../')
os.getcwd()

'/home/ubuntu/msc-project-speaker-recognition/msc-project'

In [2]:
import argparse
import visdom
import numpy as np
from tqdm import tqdm

import torch
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

from data import NpzFolder, NpzLoader, TBPTTIter
from model import Loop, MaskedMSE
from utils import create_output_dir, wrap, check_grad

import matplotlib.pyplot as plt

import speaker_recognition as sr

import pandas as pd

import notebook_utils as nu

%load_ext autoreload
%autoreload 2

/usr/local/lib/python2.7/dist-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
#checkpoint = 'models/vctk/bestmodel.pth'

# model checkpoint
checkpoint_file = '/home/ubuntu/loop/checkpoints/vctk-16khz-cmu-no-boundaries-all-noise-2/bestmodel.pth'
#checkpoint_file = 'checkpoints/vctk-16khz-cmu-no-boundaries-all/lastmodel.pth'

# data for this model
data='/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all'

# WORLD feature normalisation data
norm_path = '/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/norm_info/norm.dat'

# output embeddings to a file
output_file = '/tmp/embedding_file.npy'

In [4]:
# start this, need to move it to a class but haven't done it yet...
class SpeakerRecognition(object):
    def __init__(self, checkpoint='checkpoints/meanpool/lastmodel.pth', nspk=108, seq_len=300)    
        weights = torch.load(checkpoint, map_location=lambda storage, loc: storage)        
        opt = torch.load(os.path.dirname(checkpoint) + '/args.pth')
        
        self.net = RecognitionNet(seq_len, nspk)

SyntaxError: invalid syntax (<ipython-input-4-18ce88612c4f>, line 2)

### Reload speaker recognition model from checkpoint

In [35]:
checkpoint='/home/ubuntu/msc-project-speaker-recognition/msc-project/checkpoints/checkpoints/checkpoints/checkpoints/checkpoints/max_20180629/bestmodel.pth'  
weights = torch.load(checkpoint, map_location=lambda storage, loc: storage)        
opt = torch.load(os.path.dirname(checkpoint) + '/args.pth')
train_args = opt[0]

In [36]:
net = sr.RecognitionNet(seq_len=300, nspk=108)

In [37]:
net.load_state_dict(weights)
net.cuda()
net.eval()

RecognitionNet(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  (conv4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (bn4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  (conv5): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (bn5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  (fc1): Linear(in_features=1696, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=108, bias=True)
)

In [39]:
train_loader, valid_loader = sr.get_loaders(data_path='/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/')

In [40]:
criterion = nn.CrossEntropyLoss().cuda()
sr.evaluate(net, criterion, valid_loader)

Evaluation (loss 0.06) epoch 1: 100%|██████████| 68/68 [00:07<00:00,  8.75it/s]


(0.12100303454002749,
 0.9695004803073968,
 array([17, 14, 22, ..., 21, 62, 23]),
 array([17, 14, 22, ..., 21, 62, 30]),
 array([ True,  True,  True, ...,  True,  True, False], dtype=bool))

In [44]:
spkr_id = 17
text = 'How trying to stay cool could make the world even hotter'
#text = 'two million player ratings were awarded by users on the website'
npz = ''

num_spkr = 108
best_guess = np.zeros(num_spkr)
for i, spkr_id in enumerate(range(num_spkr)):
    loop_dict = nu.generate_sample_with_loop(spkr_id=spkr_id,
                                         npz=npz,
                                         text=text,
                                         checkpoint=checkpoint_file,
                                         output_dir='./',
                                         npz_path='/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/numpy_features_valid')

    output = net(Variable(torch.from_numpy(loop_dict['loop_feat'].reshape(1,1,loop_dict['loop_feat'].shape[0],63))).cuda())
    best_guess[i] = output.cpu().data.numpy().argmax()
    
    print i

./
How_trying_to_stay_cool_could_make_the_world_even_hotter
/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/norm_info/norm.dat
0
./
How_trying_to_stay_cool_could_make_the_world_even_hotter
/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/norm_info/norm.dat
1
./
How_trying_to_stay_cool_could_make_the_world_even_hotter
/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/norm_info/norm.dat
2
./
How_trying_to_stay_cool_could_make_the_world_even_hotter
/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/norm_info/norm.dat
3
./
How_trying_to_stay_cool_could_make_the_world_even_hotter
/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/norm_info/norm.dat
4
./
How_trying_to_stay_cool_could_make_the_world_even_hotter
/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/norm_info/norm.dat
5
./
How_trying_to_stay_cool_could_make_the_world_even_hotter
/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/norm_info/norm.dat
6
./
How_trying_to_stay_cool_could_make_the

./
How_trying_to_stay_cool_could_make_the_world_even_hotter
/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/norm_info/norm.dat
60
./
How_trying_to_stay_cool_could_make_the_world_even_hotter
/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/norm_info/norm.dat
61
./
How_trying_to_stay_cool_could_make_the_world_even_hotter
/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/norm_info/norm.dat
62
./
How_trying_to_stay_cool_could_make_the_world_even_hotter
/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/norm_info/norm.dat
63
./
How_trying_to_stay_cool_could_make_the_world_even_hotter
/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/norm_info/norm.dat
64
./
How_trying_to_stay_cool_could_make_the_world_even_hotter
/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/norm_info/norm.dat
65
./
How_trying_to_stay_cool_could_make_the_world_even_hotter
/home/ubuntu/loop/data/vctk-16khz-cmu-no-boundaries-all/norm_info/norm.dat
66
./
How_trying_to_stay_cool_could_m

In [45]:
1.0*(best_guess == range(num_spkr)).sum() / num_spkr

0.8240740740740741